In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_train = pd.read_csv('../data/raw/train.csv.zip')
df_test = pd.read_csv('../data/raw/test.csv.zip')

In [3]:
df_all = pd.concat([df_train, df_test])

In [4]:
df_all.head()

,ID,aesthetic_sub_line,color,en_US_description,fr_FR_price,function,macro_function,macro_material,model,month,product_gender,product_type,sku_hash,sub_function,target
0,3d8a4ae769b526187c36901f204691a663333fa4_1,DAMIER AZUR,AZUR,"Fashioned from summery Damier Azur canvas, thi...",370.0,SMALL LEATHER GOODS,LG ACCESSORIES,CANVAS,PF.VICTORINE,1,Women,Leather Goods,3d8a4ae769b526187c36901f204691a663333fa4,COMPACT WALLETS,1366.0
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,SANS LIGNE ESTHETIQUE,NaN,This collectible piece features our iconic Pet...,295.0,BAG CHARMS,FANCY ACCESSORIES,CANVAS,BAG CHARM PETITE MALLE,1,Women,Accessories,c05a54f7067be054ec4b27d0d6081353ef7d9df6,BAG CHARMS,526.0
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,MAHINA,GALET,"The perfect summer companion, this effortlessl...",2660.0,DAILY BAGS,CITY BAGS,LEATHER,GIROLATA,1,Women,Leather Goods,7cb4d3626bd48a9b523d8693266219c34aeccde8,SHOULDER BAGS,503.0
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,MAHINA,MAGNOLIA,"The perfect summer companion, this effortlessl...",2660.0,DAILY BAGS,CITY BAGS,LEATHER,GIROLATA,1,Women,Leather Goods,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,SHOULDER BAGS,276.0
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,SANS LIGNE ESTHETIQUE,NaN,This distinctive charm necklace reinterprets d...,325.0,NECKLACES,FASHION JEWELRY,NON PRECIOUS METAL,COLL.CHARMS FOR GENTLEMEN,1,Men,Accessories,6913a128945e0efeafc52101dcdeaa610eaa4430,NECKLACES,206.0


In [5]:
df_image_vecs = pd.read_csv('../data/raw/vimages.csv.zip')
df_navi = pd.read_csv('../data/raw/vimages.csv.zip')
df_sales = pd.read_csv('../data/raw/sales.csv.zip')

In [6]:
df_sales.columns

Index(['Date', 'day_transaction_date', 'Month_transaction', 'type',
       'zone_number', 'country_number', 'name', 'sku_hash', 'sales_quantity',
       'currency_rate_USD', 'currency_rate_GBP', 'currency_rate_CNY',
       'currency_rate_JPY', 'currency_rate_KRW',
       'currency_rate_USD_1_day_before', 'currency_rate_GBP_1_day_before',
       'currency_rate_CNY_1_day_before', 'currency_rate_JPY_1_day_before',
       'currency_rate_KRW_1_day_before', 'currency_rate_USD_2_day_before',
       'currency_rate_GBP_2_day_before', 'currency_rate_CNY_2_day_before',
       'currency_rate_JPY_2_day_before', 'currency_rate_KRW_2_day_before',
       'currency_rate_USD_3_day_before', 'currency_rate_GBP_3_day_before',
       'currency_rate_CNY_3_day_before', 'currency_rate_JPY_3_day_before',
       'currency_rate_KRW_3_day_before', 'currency_rate_USD_4_day_before',
       'currency_rate_GBP_4_day_before', 'currency_rate_CNY_4_day_before',
       'currency_rate_JPY_4_day_before', 'currency_rate_KRW_

In [7]:
df_sum = df_sales.groupby(['sku_hash']).sum().reset_index()
df_mean = df_sales.groupby([ 'sku_hash']).mean().reset_index()
df_count = df_sales.groupby([ 'sku_hash']).agg('count').reset_index()
df_max = df_sales.groupby([ 'sku_hash']).agg('max').reset_index()
df_min = df_sales.groupby([ 'sku_hash']).agg('min').reset_index()
df_std = df_sales.groupby([ 'sku_hash']).std().reset_index()

In [8]:
df_new = df_all.copy()

In [9]:
dfs = [df_sum, df_mean, df_count, df_max, df_min, df_std]
names = ['sum', 'mean', 'count', 'max', 'min', 'std']
for i in range(len(dfs)):
    tmp_df = dfs[i]
    tmp_df.rename(columns={'Month_transaction':"month"}, inplace=True)
    df_new = df_new.merge(tmp_df, on=['sku_hash'], how='left',  suffixes=["", names[i]] )

In [10]:
for tmp_df in dfs:
    del(tmp_df)

In [11]:
df_all = df_new.copy()

In [12]:
df_baseline = pd.read_csv('../data/baselined_dataset.csv')[['sales_quantity_log', 'ID']]

In [13]:
print(df_baseline.shape)

(10464, 2)


In [14]:
new_features = set(df_baseline.columns.tolist()) - set(df_all.columns.tolist())
new_features = list(new_features) + ['ID']

In [15]:
df_all = df_all.merge(df_baseline[new_features], on='ID', how='left')

## TF-IDF 

In [16]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [17]:
def parser(data):
    corpus = []
    for i in range(0, len(data)):
        review = re.sub('[^a-zA-Z]', ' ', data['en_US_description'][i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [18]:
vectorizer = TfidfVectorizer(min_df=3,  max_features=10000,
        strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        stop_words = 'english')

In [19]:
vec_tfidf = vectorizer.fit_transform(parser(df_all)).toarray()

In [20]:
SVD_FEATURES = 120
svd = TruncatedSVD(n_components=SVD_FEATURES)
svd.fit(vec_tfidf)

TruncatedSVD(algorithm='randomized', n_components=120, n_iter=5,
       random_state=None, tol=0.0)

In [21]:
vec_tfidf = svd.transform(vec_tfidf)
vec_tfidf = pd.DataFrame(vec_tfidf , columns=['svd_{}'.format(i) for i in range(SVD_FEATURES)])

In [22]:
df_all = pd.concat((df_all, vec_tfidf), axis=1)

In [23]:
df_all.shape

(10464, 700)

## Navidation data

In [24]:
df_navi = pd.read_csv('../data/raw/navigation.csv.zip')

In [25]:
df_navi = df_navi.groupby('sku_hash').agg({"page_views":"sum", "addtocart":'sum'}).reset_index()
df_navi['page_views'] = df_navi['page_views'].apply(np.log1p)
df_navi['conversion_rate'] = df_navi['addtocart'] / df_navi['page_views']
df_all = df_all.merge(df_navi, on='sku_hash', how='left')

## LighGBM

In [26]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [27]:
cat_features = df_all.columns[df_all.dtypes.apply(lambda x: x!=float)].tolist()
cat_features.remove('ID')
cat_features.remove('en_US_description')
cat_features.remove('sku_hash')

cat_features

['aesthetic_sub_line',
 'color',
 'function',
 'macro_function',
 'macro_material',
 'model',
 'month',
 'product_gender',
 'product_type',
 'sub_function',
 'monthsum',
 'zone_number',
 'country_number',
 'sales_quantity',
 'Date',
 'day_transaction_date',
 'monthcount',
 'type',
 'zone_numbercount',
 'country_numbercount',
 'name',
 'sales_quantitycount',
 'currency_rate_USDcount',
 'currency_rate_GBPcount',
 'currency_rate_CNYcount',
 'currency_rate_JPYcount',
 'currency_rate_KRWcount',
 'currency_rate_USD_1_day_beforecount',
 'currency_rate_GBP_1_day_beforecount',
 'currency_rate_CNY_1_day_beforecount',
 'currency_rate_JPY_1_day_beforecount',
 'currency_rate_KRW_1_day_beforecount',
 'currency_rate_USD_2_day_beforecount',
 'currency_rate_GBP_2_day_beforecount',
 'currency_rate_CNY_2_day_beforecount',
 'currency_rate_JPY_2_day_beforecount',
 'currency_rate_KRW_2_day_beforecount',
 'currency_rate_USD_3_day_beforecount',
 'currency_rate_GBP_3_day_beforecount',
 'currency_rate_CNY_3_day

In [28]:
numeric_features = df_all.columns[df_all.dtypes.apply(lambda x: x==float)].tolist()

In [29]:
target = 'target'

In [30]:
numeric_features.remove(target)

In [31]:
features = cat_features + numeric_features

## transrform target

In [32]:
df_all[target] = df_all[target].apply(np.log)

## Transform features 

In [33]:
for cat_feature in cat_features:
    df_all[cat_feature] = df_all[cat_feature].astype('category')

In [34]:
df_train = df_all[df_all[target].notnull()]
df_test = df_all[df_all[target].isnull()]

X_train, X_test,  y_train, y_test = train_test_split(df_train[features], df_train[target], test_size=0.1)

In [35]:
X_train.shape

(4768, 699)

In [36]:
gbm = lgb.LGBMRegressor(colsample_bytree=0.6,
       importance_type='split', learning_rate=0.01, max_depth=-1,
       min_child_samples=36, min_child_weight=0.1, min_split_gain=0.0,
       n_estimators=4022, n_jobs=-1, num_leaves=800, objective='rmse',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
       subsample=0.9, subsample_for_bin=200000, subsample_freq=2)


In [37]:
gbm.fit(df_train[features],df_train[target],
        eval_set=[(X_train, y_train) ,(X_test, y_test)],
        eval_metric='rmse',
        early_stopping_rounds=100, verbose=100)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.577535	valid_1's rmse: 0.604606
[200]	valid_0's rmse: 0.393171	valid_1's rmse: 0.421895
[300]	valid_0's rmse: 0.303152	valid_1's rmse: 0.329431
[400]	valid_0's rmse: 0.249227	valid_1's rmse: 0.273117
[500]	valid_0's rmse: 0.214305	valid_1's rmse: 0.236307
[600]	valid_0's rmse: 0.189544	valid_1's rmse: 0.209616
[700]	valid_0's rmse: 0.173483	valid_1's rmse: 0.191878
[800]	valid_0's rmse: 0.161004	valid_1's rmse: 0.178131
[900]	valid_0's rmse: 0.149601	valid_1's rmse: 0.165854
[1000]	valid_0's rmse: 0.139838	valid_1's rmse: 0.155186
[1100]	valid_0's rmse: 0.132291	valid_1's rmse: 0.147197
[1200]	valid_0's rmse: 0.126097	valid_1's rmse: 0.140327
[1300]	valid_0's rmse: 0.119241	valid_1's rmse: 0.132869
[1400]	valid_0's rmse: 0.112826	valid_1's rmse: 0.125914
[1500]	valid_0's rmse: 0.106912	valid_1's rmse: 0.119754
[1600]	valid_0's rmse: 0.102601	valid_1's rmse: 0.114977
[1700]	valid_0's rmse: 0.0980524	v

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.6,
       importance_type='split', learning_rate=0.01, max_depth=-1,
       min_child_samples=36, min_child_weight=0.1, min_split_gain=0.0,
       n_estimators=4022, n_jobs=-1, num_leaves=800, objective='rmse',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
       subsample=0.9, subsample_for_bin=200000, subsample_freq=2)

In [38]:
#df_all.to_csv('../data/second_df.csv', index=None)

In [39]:
preds = gbm.predict(df_test[features], num_iteration=gbm.best_iteration_)

In [40]:
df_explore = df_train[['target']]
df_explore['pred'] = gbm.predict(df_train[features])
df_explore['pred'] = df_explore['pred'].apply(np.exp)
df_explore['target'] = df_explore['target'].apply(np.exp)

In [41]:
df_explore.sample(5)

,target,pred
2076,110.0,111.033245
1831,909.0,928.141548
1029,83.0,84.424859
1659,54.0,53.710730
1360,66.0,66.384496


In [42]:
df_sub = pd.DataFrame()
df_sub['ID'] = df_test['ID']
df_sub['target'] = preds

In [43]:
df_sub['target']= df_sub['target'].apply(np.exp)

In [44]:
df_sub.to_csv('../data/clean_submission.csv', index=False)

## Check correlation to other submissions 

In [45]:
df_prev_sub = pd.read_csv('../data/sub_only_sales.csv')
df_sub = df_sub.merge(df_prev_sub, on='ID', suffixes=["", "_only_sales"])

In [46]:
#df_sub_tf = pd.read_csv('../data/sub_tfidf_gad.csv')

In [47]:
df_sub_mean = pd.read_csv('../data/sub_3_without_log_sales.csv')
df_sub = df_sub.merge(df_sub_mean, on='ID', suffixes=["", "_no_sales|"])

In [48]:
df_sub.corr()

,target,target_only_sales,target_no_sales|
target,1.000000,0.899217,0.868403
target_only_sales,0.899217,1.000000,0.908228
target_no_sales|,0.868403,0.908228,1.000000


In [49]:
df_sub['target'] = df_sub['target']*0.3 +  df_sub['target_only_sales']*0.7

In [50]:
df_sub[['ID', 'target']].to_csv('../data/df_sub.csv', index=False)